# Audioset
After dowloading the data set this jupyter will provide all the nessecary steps needed to utilize it annd train a model with it
We highly recomend to download the dataset in a cloud because some of the files are downloaded in their full size and we cut them later.
Whetever we provide in dowload section should be skipped if the dataset is already download and cutted into 10 sec windows.

# Dowload process

In [11]:
"""

Essential Procerdures for the dowload to work


"""
import os
directory = 'C:\\Users\\giorg\\OneDrive\\Υπολογιστής\\DL Project'
os.chdir(directory)

from Scripts import Utilities as ut
from Scripts import Feature_Extraction as fe
from Scripts import Model_Training as mt
from Scripts import data_loading as dl
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense, Dropout



dataset = "audioset"





In [ ]:
# Clone the audioset repository
!git clone https://github.com/lukefahr/audioset.git
%cd audiosetAC
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz

In [ ]:
from Scripts import Audioset_Parser as ap

When cloning the directory o folder which is called metadat will be finoun inside it the are the "balanced_train_segments "and "eval_segments" csv need to dowload the files and cut them from Youtube

In [ ]:
ad.download_and_cut_audioset("segments_csv", "audioset_dowload_directory")

After download you need to flagg files which where created before the date you started dowloading the dataset , this is the way we found to discriminate which files where correctly cuted and what where not. Then you move these files in another directory and from them you cut them based on the csv again localy

In [ ]:
""""
All paths are example paths change them with oyur own this is the pipeline of the procedure we follow and you oly need to dowload the Audioset Train and Eval

"""

cutoff_date = datetime(2024, 6, 6).timestamp() # Example date
folder_path = "audioset_dowload_directory"

ut.flag_old_files(folder_path, cutoff_date)

ut.move_files_with_prefix(folder_path,"flaged_files_dest", "FLAG-")

ut.delete_files_with_prefix("flaged_files_dest","FLAG-")

ap.cut_audio( "train_segments_csv","flaged_files_dest", "AS_Train") # Change for eval\

ap.convert_audio(folder_path, "AS_Train")



# Extract Labels

After you downloaded AS_Train and AS_Eval you need to extract the labels for the csv which is located at the ontology.json which is in git hub directory we mounted at the start "balanced_train_segments.csv" and "eval_Segments.csv" is also there but we will provide them loccally ath the "Audioset\\CSVs" destination.

------ Before starting got to both csv and add as columns class 1,class 2,class 3,.......,class_12,class 13----------------
We have done it here but if you want to replicate from scratch you need to do it manually

In [ ]:
input_csv = "Audioset\\CSVs\\balanced_train_segments.csv"
ontology_json = "Audioset\\CSVs\\ontology.json"
output_csv= "Audioset\\CSVs\\balanced_train_labels.csv"
sound_path = "Audioset\\AS_Train"

ap.create_labels(input_csv, ontology_json, output_csv)
# Because we created the function to work with csv we put the same output p[ath as input path so it will overwrite the file
ap.labels_to_use(sound_path, output_csv, output_csv)
# Seperate files into class fodler
ut.move_to_class_folder("Audioset\CSVs\GPTTrain.csv","Audioset\AS_Train",dataset)

In [ ]:
input_csv = "Audioset\CSVs\eval_segments.csv"
ontology_json = "Audioset\\CSVs\\ontology.json"
output_csv= "Audioset\\CSVs\\eval_labels.csv"
sound_path = "Audioset\\AS_Eval"

ap.create_labels(input_csv, ontology_json, output_csv)
ap.labels_to_use(sound_path, output_csv, output_csv)

ut.move_to_class_folder("Audioset\CSVs\GPTEval.csv","Audioset\AS_Eval",dataset)

Now that we have the csv with the files what we tried to do is to aplly NLP methods and clustering to cluster files that had similar names to 8 categories whatever we did was nothing compared to what GPT-4o gave us , to put it in prespective from 20% accuracy it skyrocketed to 50% so it would a shame not to use it, nevertheless we include this in the DL_Project in Bonus folder because we think it was a quite hinest try. We will go on with the gpt labels which are located at "DL_Project\Audioset\CSVs"


- We asked GPT to seperate our folders into 10 categories but some of them where oversampled and 1 very undersabled (140 files total) to put it prespective other folders had form 2.000 to 8.000 files so we did not include this category in training. We made othe manual modifications but we will dig into that later.

# Feature Extraction
The same method as we did in the UrbanSound8k is used here.

ATTENTION Name feature folders Train_Features, Eval_Features corespondingly

In [13]:
fe.CNN_Features_mel("Audioset\AS_Train", classes, "Audioset\\Train_Features")
fe.CNN_Features_mel("Audioset\AS_Eval", classes, "Audioset\\Eval_Features")

In [9]:
x_train, x_val, y_train, y_val =  dl.load_data("Audioset", dataset)

In [5]:
model = mt.create_model_CNN_AS()

learning_rate = 0.0001

model.compile(optimizer=Adam(learning_rate = learning_rate ), loss='categorical_crossentropy', metrics=[mt.f1_score, 'accuracy'])

early_stopping = EarlyStopping(monitor='val_f1_score', mode='max', patience=15, verbose=1)

In [ ]:
hist = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=50, callbacks = early_stopping,
    batch_size=32
    )

In [ ]:
_ = hist.history['f1_score']
c = hist.history['val_f1_score']
mt.plot_training_history(_,c,"f1_score")

In [ ]:
mt.plot_confusion_matrix(model,x_val,y_val,classes)

In [ ]:
model.save("Models\\model_name.h5")